In [1]:
import json

In [2]:
file = '../data_resources/topics/vng_topics.json'
topic_json = json.load(open(file))

In [3]:
for obj in topic_json['topics']:
    print(obj['name'])
    
    for sub in obj['sub']:
        print('- {}'.format(sub['name']))

Arbeidsvoorwaarden en personeelsbeleid
- Arbeidsomstandigheden
- Arbeidsvoorwaarden
- Cao-onderhandelingen
- Collectieve zorgverzekering
- Brandweerpersoneel en FLO-overgangsrecht
- H3 en beloningsbeleid
- Individueel keuzebudget (IKB)
- Integriteit
- Normalisering rechtspositie ambtenaren
- P&O-beleid
- Rechtspositie politieke ambtsdragers
- Sociale zekerheid
- Werk voor arbeidsbeperkten
Asiel
- Asielbeleid en integratie
Belastingen
- Fiscaal procesrecht
- Gemeentelijk belastinggebied
- Heffing
- Invordering
- Leges
- Overige belastingen
- OZB
- Parkeerbelastingen
- Rijksbelastingen
- Samenwerking belastingen
- Wet WOZ
- Vennootschapsbelasting (Vpb) voor overheden
Bestuur
- Gemeentelijke samenwerking
- Herindeling
- Versterking lokale democratie
- Verkiezingen & referenda
Cultuur en sport
- Architectuur en erfgoed
- Bibliotheekwerk
- Combinatiefuncties
- Cultuur, kunst en kunstenaars
- Cultuureducatie en amateurkunst
- Film, media en kabel
- Sport
Dienstverlening en informatiebeleid
-

In [4]:
# Gensim
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format("../data_resources/embeddings/cow-320.txt")
katvec = model['kat']
model.most_similar('kat')

[('hond', 0.8295692205429077),
 ('poes', 0.7784526348114014),
 ('fret', 0.7531309127807617),
 ('konijn', 0.7332630157470703),
 ('cavia', 0.7052830457687378),
 ('pup', 0.7040694952011108),
 ('huiskat', 0.7027560472488403),
 ('katten', 0.6984586119651794),
 ('hamster', 0.6966392993927002),
 ('hondje', 0.6850977540016174)]

In [5]:
import numpy as np

class DocSim(object):
    def __init__(self, w2v_model , stopwords=[]):
        self.w2v_model = w2v_model
        self.stopwords = stopwords

    def vectorize(self, doc):
        """Identify the vector values for each word in the given document"""
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector


    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=[], threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({
                    'score' : sim_score,
                    'doc' : doc
                })
            # Sort results by score in desc order
            results.sort(key=lambda k : k['score'] , reverse=True)

        return results

In [6]:
ds = DocSim(model)

In [7]:
source_doc = 'how to delete an invoice'
target_docs = ['delete a invoice', 'how do i remove an invoice', 'purge an invoice']

# This will return 3 target docs with similarity score
sim_scores = ds.calculate_similarity(source_doc, target_docs)

print(sim_scores)

[{'score': 0.9552662, 'doc': 'how do i remove an invoice'}, {'score': 0.8816773, 'doc': 'purge an invoice'}, {'score': 0.84273493, 'doc': 'delete a invoice'}]


In [16]:
for obj in topic_json['topics']:
    print(obj['name'])
    
    for sub in obj['sub']:
        print('- {}'.format(sub['name']))
        
        for comp in obj['sub']:
            print('-- vs {}: {}'.format(comp['name'], model.wmdistance(sub['description'], comp['description'])))

Arbeidsvoorwaarden en personeelsbeleid
- Arbeidsomstandigheden
-- vs Arbeidsomstandigheden: 0.0
-- vs Arbeidsvoorwaarden: 0.08625038636979283
-- vs Cao-onderhandelingen: 0.18525376313399713
-- vs Collectieve zorgverzekering: 0.09809261907666679
-- vs Brandweerpersoneel en FLO-overgangsrecht: 0.08831344980595007
-- vs H3 en beloningsbeleid: 0.13211305063697917
-- vs Individueel keuzebudget (IKB): 0.11878789533670668
-- vs Integriteit: 0.08684135198896172
-- vs Normalisering rechtspositie ambtenaren: 0.11642184587402091
-- vs P&O-beleid: 0.15824532546962244
-- vs Rechtspositie politieke ambtsdragers: 0.11202198362015518
-- vs Sociale zekerheid: 0.09443543203524564
-- vs Werk voor arbeidsbeperkten: 0.1306770115750274
- Arbeidsvoorwaarden
-- vs Arbeidsomstandigheden: 0.08625038636979283
-- vs Arbeidsvoorwaarden: 0.0
-- vs Cao-onderhandelingen: 0.16515912672080102
-- vs Collectieve zorgverzekering: 0.12325707020819568
-- vs Brandweerpersoneel en FLO-overgangsrecht: 0.10396562857920495
-- vs

-- vs Collectieve zorgverzekering: 0.16418075997584677
-- vs Brandweerpersoneel en FLO-overgangsrecht: 0.13996518993886198
-- vs H3 en beloningsbeleid: 0.16361092231718374
-- vs Individueel keuzebudget (IKB): 0.14568968650509073
-- vs Integriteit: 0.13080232122208263
-- vs Normalisering rechtspositie ambtenaren: 0.1324328008027745
-- vs P&O-beleid: 0.162284274091893
-- vs Rechtspositie politieke ambtsdragers: 0.17984032230052405
-- vs Sociale zekerheid: 0.14764373973934514
-- vs Werk voor arbeidsbeperkten: 0.0
Asiel
- Asielbeleid en integratie
-- vs Asielbeleid en integratie: 0.0
Belastingen
- Fiscaal procesrecht
-- vs Fiscaal procesrecht: 0.0
-- vs Gemeentelijk belastinggebied: 0.15013882939640696
-- vs Heffing: 0.15488019738757888
-- vs Invordering: 0.16021386329394463
-- vs Leges: 0.1313922247004681
-- vs Overige belastingen: 0.22154389162326005
-- vs OZB: 0.1808061859620949
-- vs Parkeerbelastingen: 0.18112392163487911
-- vs Rijksbelastingen: 0.16815986988063347
-- vs Samenwerking 

-- vs Bibliotheekwerk: 0.0
-- vs Combinatiefuncties: 0.18664005771895925
-- vs Cultuur, kunst en kunstenaars: 0.24398047559176972
-- vs Cultuureducatie en amateurkunst: 0.19450760183138707
-- vs Film, media en kabel: 0.17278075641693247
-- vs Sport: 0.19506094652421446
- Combinatiefuncties
-- vs Architectuur en erfgoed: 0.22305636257827394
-- vs Bibliotheekwerk: 0.18664005771895925
-- vs Combinatiefuncties: 0.0
-- vs Cultuur, kunst en kunstenaars: 0.1601618589338
-- vs Cultuureducatie en amateurkunst: 0.1574635704387112
-- vs Film, media en kabel: 0.20911232271898159
-- vs Sport: 0.12787298837360841
- Cultuur, kunst en kunstenaars
-- vs Architectuur en erfgoed: 0.2018779750692817
-- vs Bibliotheekwerk: 0.24398047559176989
-- vs Combinatiefuncties: 0.1601618589338
-- vs Cultuur, kunst en kunstenaars: 0.0
-- vs Cultuureducatie en amateurkunst: 0.15187428597554267
-- vs Film, media en kabel: 0.24532473561094228
-- vs Sport: 0.20646325070736607
- Cultuureducatie en amateurkunst
-- vs Archi

-- vs Gemeentefonds en BTW-compensatiefonds: 0.166960572879489
-- vs Gemeenten & kapitaalmarkt: 0.0
-- vs Gemeentefinanciën in beleidsdossiers: 0.1548998462449893
- Gemeentefinanciën in beleidsdossiers
-- vs Begroting en verantwoording: 0.2162809572509182
-- vs Financiële positie gemeenten: 0.16678202062549363
-- vs Gemeentefonds en BTW-compensatiefonds: 0.17947883071928075
-- vs Gemeenten & kapitaalmarkt: 0.1548998462449896
-- vs Gemeentefinanciën in beleidsdossiers: 0.0
Internationaal
- Gemeentelijk internationaal beleid
-- vs Gemeentelijk internationaal beleid: 0.0
Jeugd
- Jeugdbescherming en Jeugdreclassering
-- vs Jeugdbescherming en Jeugdreclassering: 0.0
-- vs Child Friendly Cities: 0.1504176617051697
-- vs Jeugdgezondheidszorg: 0.1579982982168877
-- vs Jeugdhulp: 0.09499036788461902
-- vs Kindcentra: 0.12594635820230005
-- vs Kinderopvang: 0.10654838327544801
-- vs Zorglandschap specialistische jeugdhulp: 0.07026944688423183
- Child Friendly Cities
-- vs Jeugdbescherming en Jeu

-- vs Afval: 0.15123742562910752
-- vs Afval: 0.15123742562910752
-- vs Bodem en ondergrond: 0.0
-- vs Geluidhinder: 0.19417766307974055
-- vs Mobiliteitsbeleid: 0.19972695730047985
-- vs Luchtkwaliteit: 0.17161139853860655
-- vs Landbouw en veehouderij: 0.11951505489129828
-- vs Energie en klimaat: 0.15867512940056133
-- vs Vergunningverlening, toezicht en handhaving: 0.16252117022500917
-- vs Water en riolering: 0.14920044982985242
- Geluidhinder
-- vs Asbest: 0.22691188929125108
-- vs Afval: 0.2261574299337271
-- vs Afval: 0.2261574299337271
-- vs Bodem en ondergrond: 0.19417766307974085
-- vs Geluidhinder: 0.0
-- vs Mobiliteitsbeleid: 0.303352059436708
-- vs Luchtkwaliteit: 0.1809591904894294
-- vs Landbouw en veehouderij: 0.18982223266263246
-- vs Energie en klimaat: 0.22169076980888058
-- vs Vergunningverlening, toezicht en handhaving: 0.21064592463768325
-- vs Water en riolering: 0.2464269681821783
- Mobiliteitsbeleid
-- vs Asbest: 0.18811813473525316
-- vs Afval: 0.154146879730

-- vs Wet openbaarheid van bestuur (Wob): 0.2031640380948295
-- vs Awb: 0.1813093720930513
-- vs APV: 0.1611146186064533
-- vs Aanbesteden: 0.1339603845064932
-- vs Gemeentewet: 0.0
-- vs Modelverordeningen: 0.17051163946589276
-- vs Rechtspersonen: 0.17043677429826332
-- vs Subsidierecht: 0.15242076117899686
-- vs Overheidsaansprakelijkheid: 0.13522148606955542
-- vs Overeenkomsten: 0.0950165892787392
- Modelverordeningen
-- vs Winkeltijdenwet: 0.11786369657501548
-- vs Wet openbaarheid van bestuur (Wob): 0.2099029777744895
-- vs Awb: 0.17361507718807426
-- vs APV: 0.10389499139459303
-- vs Aanbesteden: 0.12072872280915543
-- vs Gemeentewet: 0.17051163946589307
-- vs Modelverordeningen: 0.0
-- vs Rechtspersonen: 0.19558621233238124
-- vs Subsidierecht: 0.15123432761741498
-- vs Overheidsaansprakelijkheid: 0.2271774691480414
-- vs Overeenkomsten: 0.17827250102115555
- Rechtspersonen
-- vs Winkeltijdenwet: 0.20812228288719642
-- vs Wet openbaarheid van bestuur (Wob): 0.2102406795764892


-- vs Wonen, begeleiding en zorg: 0.10900559098304795
-- vs RO wet- en regelgeving: 0.1303931586482319
-- vs Woonbeleid: 0.11690585457822067
- Wabo, Omgevingsvergunning
-- vs Grondzaken/-beleid: 0.1447337507501757
-- vs Bouwregelgeving: 0.09319418920889966
-- vs Bevolkingsdaling: 0.17942756820870626
-- vs Huisvestingswet: 0.12173295736510104
-- vs Kabels en leidingen: 0.2105342495570692
-- vs Leegstand, transformatie en herbestemming: 0.14530930262362193
-- vs Landschap en natuur: 0.19612066270483253
-- vs Landelijk gebied: 0.2096401914045474
-- vs Omgevingswet: 0.13159553019946782
-- vs Wabo, Omgevingsvergunning: 0.0
-- vs Wonen, begeleiding en zorg: 0.17438892860591046
-- vs RO wet- en regelgeving: 0.15493240752588008
-- vs Woonbeleid: 0.15929934429844064
- Wonen, begeleiding en zorg
-- vs Grondzaken/-beleid: 0.15346502303104317
-- vs Bouwregelgeving: 0.1520707422826785
-- vs Bevolkingsdaling: 0.18154611250555636
-- vs Huisvestingswet: 0.14102037432962264
-- vs Kabels en leidingen: 0

-- vs Cyberveiligheid: 0.14085496582121496
-- vs Veilig ondernemen: 0.17330347768461724
-- vs Veiligheidshuizen: 0.0863048204977453
-- vs Veiligheidsregio's: 0.15002249513610955
-- vs Softdrugsbeleid: 0.12492433123181217
-- vs Aanpak radicalisering: 0.08106993569354216
-- vs Prostitutie: 0.13012125860992996
-- vs Drank- en Horecawet: 0.16893845192711748
-- vs Ex-gedetineerden: 0.14838327286457798
-- vs Politie: 0.0
-- vs Ondermijning: 0.1256362735429134
-- vs Openbare ordebevoegdheden: 0.13030058692894994
-- vs Kernbeleid Veiligheid: 0.16667354742800589
- Ondermijning
-- vs Cyberveiligheid: 0.15362386011557894
-- vs Veilig ondernemen: 0.14279172373290408
-- vs Veiligheidshuizen: 0.11271088092769097
-- vs Veiligheidsregio's: 0.1824415281518975
-- vs Softdrugsbeleid: 0.12100323017080908
-- vs Aanpak radicalisering: 0.09701699178899933
-- vs Prostitutie: 0.1577410138768464
-- vs Drank- en Horecawet: 0.15899200861471421
-- vs Ex-gedetineerden: 0.1282526490668839
-- vs Politie: 0.1256362735

In [33]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer

tokenizer = RegexpTokenizer(r'\w+')
nl_stop = get_stop_words('nl')
p_stemmer = PorterStemmer()

def preprocess(sentence):
    
    # clean and tokenize document string
    raw = sentence.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in nl_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    return ' '.join(stemmed_tokens)

In [52]:
doc1 = 'Vertellen wat we hebben gedaan, dat hebben we overigens de afgelopen jaren ook consequent gedaan. Waar je bij sommige partijen ineens een opleving in activiteit ziet zo vlak voor de verkiezingen, daar kunnen wij terugkijken op een actieve communicatie via onze website en social media. Natuurlijk waren we er ook altijd bij als er een bijeenkomst was met belanghebbenden of andere geïnteresseerden. Dat kon omdat we dit belangrijk vinden en omdat we krachten goed konden verdelen in een fractie van vier raadsleden en drie commissieleden. Als grote fractie hebben we dus veel invloed uit kunnen oefenen in de afgelopen jaren. Dat willen we de komende jaren nog meer! Wat hebben we bereikt? Helaas kunnen we hier niet volledig zijn, maar de volgende punten willen we hier graag met je delen. Financiën: we hebben gezorgd voor een solide begroting, ook in de economisch moeilijke jaren. Met name door minder uit te geven.Minder regels: door ons initiatief mogen ondernemers met een winkel nu zelf bepalen of ze op zondag open gaan.Meer woningen: het voldoende bouwen van koop- en huurwoningen en het ombouwen van lege kantoorpanden was voor ons erg belangrijk. Hierbij hebben we de Houtense kwaliteit bewaakt.Veiligheid: de criminaliteit is met ca. 17%  gedaald sinds 2014, verder is er op initiatief van ons meer aandacht voor bewust brandveilig ondernemen.De lokale economie werd gestimuleerd door ontwikkeling van bedrijventerreinen en het binnenhalen van nieuwe bedrijven door onze wethouder. Wat willen we de komende jaren doen? Ook hier geen volledige lijst, al onze ambities kun je hier terugvinden. Financiën: we blijven kritisch op de uitgaven van de gemeente, ook nu het economisch beter gaat. Hierdoor kunnen we de lokale belastingen fors verlagen en meer geld steken in het onderhoud van de openbare ruimte.Wonen: behoud van het karakter van Houten en de kleine kernen, wat ons betreft dus geen grootschalige woningbouw buiten de Rondweg.Veiligheid: je moet je veilig voelen in je eigen buurt en huis in Houten. Daarom willen wij een hardere aanpak van auto-en woninginbraken en diefstal van fietsen. Verder zal uitbreiding van het camera toezicht in openbare gebieden en bij toegangswegen afschrikwekkend zijn voor criminelen.Ondernemen: ondernemers zijn onmisbaar voor werk en een bloeiende economie. Daarom is het belangrijk te zorgen voor een goede bereikbaarheid en nette uitstraling. Voor de agrarische ondernemers in het buitengebied willen we ruimte om te ondernemen, ook op het gebied van recreatie, toerisme en natuurbeheer.Duurzaamheid: dit is voor ons ook het zuinig omgaan met energie en water en verstandig omgaan met het milieu. Wij stimuleren duurzaamheid door regels te verminderen, niet door met subsidies te strooien. Wij stimuleren het plaatsen van zonnepanelen op daken en zonneweides waar mogelijk. Wij zijn tegen een nieuw windmolenpark zonder draagvlak in de omgeving. In de komende periode zullen we de verkiezingsposters ophangen, debatteren, politieke cafés houden, de straat op gaan en meer. Wil je met ons praten over onze en jouw ambities? Laat het ons weten, wij horen het graag!'
doc2 = 'van de SP gepresenteerd: “Wij zetten in op gemeentelijke thuiszorg, voorrang voor Vlaardingse woningzoekenden, verdubbeling van de duurzame energieproductie en een groot aantal andere concrete maatregelen.” Met de titel ‘Voor elkaar’ wil de SP afstand nemen van het individualisme in de samenleving, en duidelijk maken dat Vlaardingers samen moeten werken om het geheel te verbeteren. ‘Voor elkaar’ verwijst er ook naar dat Vlaardingers het samen voor elkaar hebben gekregen om de gemeente er weer financieel bovenop te krijgen en er weer geïnvesteerd kan worden in de stad. Voor de komende vier jaar stelt de SP voor bijna 10 miljoen euro aan nieuw beleid voor. De SP is tot op heden de enige partij die het programma heeft voorzien van een financiële onderbouwing. “Ons programma is financieel haalbaar, en wij laten zien wat de mogelijkheden zijn om onze keuzes te financieren. Je kunt niet én de belastingen verlagen én gratis bier uitdelen. Ik roep andere partijen op om ook met cijfers te komen zodat de kiezers niet met loze beloftes worden geconfronteerd.” Aldus Hoekstra. Een aantal belangrijke voorstellen zijn: Verdubbelen van het schoolzwemmen Jaarlijks een ijsbaan Museum gratis toegankelijk Meer evenementen in de binnenstad mogelijk maken Extra inzet op bestrijden overgewicht onder kinderen Gratis OV voor minima Renoveren van de Stadskraan Extra handhavers voor aanpak hondenpoep en zwerfvuil Omscholen werklozen Steun voor archeologische en historische vereniging'

result = []

for main in topic_json['topics']:
    
    main_doc = ''
    for sub in main['sub']:
        main_doc += sub['name']
    
    for sub in main['sub']:
        sub_doc = sub['name']
        
#         score = model.wmdistance(sub_doc, doc1) + model.wmdistance(main_doc, doc1)
        
        score = model.wmdistance(preprocess(sub_doc), preprocess(doc2)) # + model.wmdistance(preprocess(main_doc), preprocess(doc1))
        result.append({'topics' : sub['name'], 'score': score})
        
KEEP = 2

while len(result) > KEEP:
    max_dist = -1
    for obj in result:
        if obj['score'] > max_dist:
            max_dist = obj['score']
            max_obj = obj
    
    result.remove(max_obj)
    
for obj in result:
    print(obj)
        

{'topics': 'Ondernemersklimaat & dienstverlening', 'score': 0.24204014213130876}
{'topics': 'Vrijwilligerswerk & mantelzorgondersteuning', 'score': 0.21668428559627873}


In [38]:
print(doc2)
print('--')
print(preprocess(doc2))

van de SP gepresenteerd: “Wij zetten in op gemeentelijke thuiszorg, voorrang voor Vlaardingse woningzoekenden, verdubbeling van de duurzame energieproductie en een groot aantal andere concrete maatregelen.” Met de titel ‘Voor elkaar’ wil de SP afstand nemen van het individualisme in de samenleving, en duidelijk maken dat Vlaardingers samen moeten werken om het geheel te verbeteren. ‘Voor elkaar’ verwijst er ook naar dat Vlaardingers het samen voor elkaar hebben gekregen om de gemeente er weer financieel bovenop te krijgen en er weer geïnvesteerd kan worden in de stad. Voor de komende vier jaar stelt de SP voor bijna 10 miljoen euro aan nieuw beleid voor. De SP is tot op heden de enige partij die het programma heeft voorzien van een financiële onderbouwing. “Ons programma is financieel haalbaar, en wij laten zien wat de mogelijkheden zijn om onze keuzes te financieren. Je kunt niet én de belastingen verlagen én gratis bier uitdelen. Ik roep andere partijen op om ook met cijfers te komen